# README

Parameters may be defined either by range or by tolerance (absolute or relative). Units from [Pint](https://pint.readthedocs.io/en/stable/) are supported. A default `Pint.UnitRegistry()` is provided to the user as `worstcase.unit`.
* `worstcase.param.byrange(nom, lb, ub, tag="")`
* `worstcase.param.bytol(nom, tol, rel, tag="")`

Both Extreme Value and Monte Carlo analyses are supported, even in the same calculation. The underlying computation is performed on the directed graph of `Parameters`, `Derivatives`, and the single-valued functions between them. This directed graph must be an acyclic computation. Extreme Value Analysis is performed by brute force, so Monte Carlo Analysis will be faster if the directed graph couples many parameters together. When `Derivatives` are coupled together, the analysis method of one `Derivative` may be overruled by one of its descendants.
* `derive.byev(*args, tag="", **kwargs)(func)`
* `derive.bymc(*args, tag="", **kwargs)(func)`

In [16]:
%reload_ext lab_black
from pprint import pprint
from worstcase import config, param, derive, unit

## Example 1: Voltage Dividers and an Amplifier

In [2]:
config.n = 2000  # Number of Monte Carlo runs. (default: 5000)
config.sigfig = 5  # Number of significant figures to print. (default: 4)

In [20]:
# Define the parameters of interest. See the diagram above.
VCC = param.byrange(nom=5 * unit.V, lb=4.8 * unit.V, ub=5.1 * unit.V, tag="vcc")
R1 = param.bytol(nom=10 * unit.kohm, tol=0.10, rel=True, tag="r1")
R2 = param.bytol(nom=10 * unit.kohm, tol=0.10, rel=True, tag="r2")

VIN = param.byrange(nom=12 * unit.V, lb=11.9 * unit.V, ub=12.05 * unit.V, tag="vin")
R3 = param.bytol(nom=50 * unit.kohm, tol=0.05, rel=True, tag="r3")
R4 = param.bytol(nom=20 * unit.kohm, tol=0.05, rel=True, tag="r4")

GAIN = param.bytol(nom=1.1, tol=0.1, rel=False, tag="gain")
R5 = param.bytol(nom=10 * unit.kohm, tol=0.01, rel=True, tag="r5")
R6 = param.bytol(nom=10 * unit.kohm, tol=0.01, rel=True, tag="r6")

pprint([VCC, VIN, GAIN, R1, R2, R3, R4, R5, R6])

[vcc: 5 V (nom), 4.8 V (lb), 5.1 V (ub),
 vin: 12 V (nom), 11.9 V (lb), 12.05 V (ub),
 gain: 1.1 (nom), 1 (lb), 1.2 (ub),
 r1: 10 kΩ (nom), 9 kΩ (lb), 11 kΩ (ub),
 r2: 10 kΩ (nom), 9 kΩ (lb), 11 kΩ (ub),
 r3: 50 kΩ (nom), 47.5 kΩ (lb), 52.5 kΩ (ub),
 r4: 20 kΩ (nom), 19 kΩ (lb), 21 kΩ (ub),
 r5: 10 kΩ (nom), 9.9 kΩ (lb), 10.1 kΩ (ub),
 r6: 10 kΩ (nom), 9.9 kΩ (lb), 10.1 kΩ (ub)]


In [21]:
# Define a generic voltage divider function.
def voltage_divider(vtop, vbot, rtop, rbot):
    return vbot + rbot * (vtop - vbot) / (rtop + rbot)

In [22]:
# Define input voltage divider derived parameters.
VNEG = derive.byev(vtop=VCC, vbot=0, rtop=R1, rbot=R2, tag="vneg")(voltage_divider)
VPOS = derive.byev(vtop=VIN, vbot=0, rtop=R3, rbot=R4, tag="vpos")(voltage_divider)

pprint([VNEG, VPOS])

[vneg: 2.5 V (nom), 2.16 V (lb), 2.805 V (ub),
 vpos: 3.4286 V (nom), 3.1622 V (lb), 3.6942 V (ub)]


In [23]:
# Define the amplifier output derived parameter.
@derive.byev(vp=VPOS, vm=VNEG, gain=GAIN, tag="amp")
def AMP(vp, vm, gain):
    return gain * (vp - vm)


print(AMP)

amp: 1.0214 V (nom), 357.24 mV (lb), 1.841 V (ub)


In [24]:
# Define the output voltage divider derived parameter.
VOUT = derive.byev(vtop=VCC, vbot=AMP, rtop=R5, rbot=R6, tag="vout")(voltage_divider)

print(VOUT)

vout: 3.0107 V (nom), 2.6397 V (lb), 3.4066 V (ub)


In [25]:
# Confirm with manual calculation.

vneg_min = VCC.lb * R2.lb / (R1.ub + R2.lb)
assert vneg_min == VNEG.lb

vpos_max = VIN.ub * R4.ub / (R3.lb + R4.ub)
assert vpos_max == VPOS.ub

amp_max = GAIN.ub * (vpos_max - vneg_min)
assert amp_max == AMP.ub

vneg = VCC.ub * R2.lb / (R1.ub + R2.lb)
amp = GAIN.ub * (vpos_max - vneg)
vout_max = amp + (VCC.ub - amp) * R6.ub / (R5.lb + R6.ub)
assert vout_max == VOUT.ub

## Unit Conversion Support

In [15]:
# worstcase.unit is a default Pint.UnitRegistry().

In [ ]:
speed = param.bytol

In [2]:
import worstcase as wca
import numpy as np
from pprint import pprint

wca.config.n = 2000  # Number of Monte Carlo runs. (default: 5000)
wca.config.sigfig = 4  # Number of significant firues to print. (default: 3)

In [3]:
spd_initial = wca.param.bytol(nom=2, tol=0.1, rel=True, unit=wca.unit("m/s"), tag="v0")
accel = wca.param.byrange(nom=0.2, lb=0.1, ub=0.5, unit=wca.unit("m/s**2"), tag="a")
distance = wca.param.byrange(nom=1, lb=0.8, ub=1.1, unit=wca.unit.km, tag="x")

pprint([spd_initial, accel, distance])

[v0: 2 m/s (nom), 1.8 m/s (lb), 2.2 m/s (ub),
 a: 200 mm/s² (nom), 100 mm/s² (lb), 500 mm/s² (ub),
 x: 1 km (nom), 800 m (lb), 1.1 km (ub)]


In [4]:
@wca.param.ev(spd_initial, accel, distance)
def spd_final(v, a, x):
    return np.sqrt(v ** 2 + 2 * a * x)


print(spd_final)

spd_final (ev)
├── a
├── v0
└── x



In [5]:
print(spd_final())

spd_final: 20.1 m/s (nom), 12.78 m/s (lb), 33.24 m/s (ub)


In [6]:
spd_final_noaccel = spd_final(a=0 * wca.unit("m/s**2"), tag="spd_noaccel")
print(spd_final_noaccel)

spd_noaccel (ev)
├── v0
└── x



In [7]:
result = spd_final_noaccel(3 * wca.unit("m/s"), x=10 * wca.unit.m)
print(result)

3.0 meter / second


In [8]:
spd_rel = wca.param.bytol(nom=20, tol=1, rel=False, unit=wca.unit("mi/hr"), tag="vrel")


@wca.param.mc(spd_final, spd_rel)
def spd_total(vf, vr):
    return vf + vr


print(spd_total)
print(spd_total())

spd_total (mc)
├── spd_final (ev)
│   ├── a
│   ├── v0
│   └── x
└── vrel

spd_total: 29.04 m/s (nom), 21.41 m/s (lb), 42.57 m/s (ub)


In [9]:
accel_sens = spd_total.ss(accel, tag="accel-sens")
print(accel_sens)
print(accel_sens())

accel-sens (mc)
└── spd_final (ev)
    └── a

accel-sens: 29.04 m/s (nom), 23.23 m/s (lb), 40.62 m/s (ub)


In [10]:
accel_distance_sens = spd_total.ss([accel, distance], tag="accel/distance-sens")
print(accel_distance_sens)
print(accel_distance_sens())

accel/distance-sens (mc)
└── spd_final (ev)
    ├── a
    └── x

accel/distance-sens: 29.04 m/s (nom), 21.75 m/s (lb), 42.16 m/s (ub)


In [11]:
finalspd_sens = spd_total.ss(spd_final, tag="finalspd-sens")
print(finalspd_sens)
print(finalspd_sens())

finalspd-sens (mc)
└── spd_final (ev)
    ├── a
    ├── v0
    └── x

finalspd-sens: 29.04 m/s (nom), 21.72 m/s (lb), 42.17 m/s (ub)


In [12]:
relspd_sens = spd_total.ss(spd_rel, tag="relspd-sens")
print(relspd_sens)
print(relspd_sens())

relspd-sens (mc)
└── vrel

relspd-sens: 29.04 m/s (nom), 28.59 m/s (lb), 29.49 m/s (ub)


In [13]:
assert spd_total.check("[length]/[time]")
# also try: spd_total.dimensionality

In [14]:
print(spd_total.units)
# also try: spd_total.u

meter / second


In [15]:
print(spd_total().ito(wca.unit("km/hr")))
# also try: spd_total().ito_base_units()
#           spd_total().ito_reduced_units()
#           spd_total().ito_root_units()

spd_total: 104.5 km/hr (nom), 76.93 km/hr (lb), 153.1 km/hr (ub)
